In [1]:
import os
import sys
import numpy as np
import pandas as pd
import re
from pathlib import Path
import glob


cwd = os.getcwd()

# Add directories for modules and data files
dirs = [x[0] for x in os.walk(cwd)]
[sys.path.append(d) for d in dirs]
data_dir = cwd+'\\Input\\DataFiles\\' # Manual way; TODO: find a relative way to do this automatically
vegas_dir = cwd+'\\Input\\VegasOdds\\'

import elo


In [3]:
p = Path('.')
paths = list(p.glob('**/*.csv'))
paths

[WindowsPath('odds_2017_basic.csv'),
 WindowsPath('Input/DataFiles/Cities.csv'),
 WindowsPath('Input/DataFiles/ConferenceTourneyGames.csv'),
 WindowsPath('Input/DataFiles/GameCities.csv'),
 WindowsPath('Input/DataFiles/NCAATourneySeeds.csv'),
 WindowsPath('Input/DataFiles/NCAATourneySlots.csv'),
 WindowsPath('Input/DataFiles/RegularSeasonCompactResults.csv'),
 WindowsPath('Input/DataFiles/RegularSeasonDetailedResults.csv'),
 WindowsPath('Input/DataFiles/SeasonElos.csv'),
 WindowsPath('Input/DataFiles/Seasons.csv'),
 WindowsPath('Input/DataFiles/SecondaryTourneyCompactResults.csv'),
 WindowsPath('Input/DataFiles/SecondaryTourneyTeams.csv'),
 WindowsPath('Input/DataFiles/TeamCoaches.csv'),
 WindowsPath('Input/DataFiles/Teams.csv'),
 WindowsPath('Input/DataFiles/TeamSpellings.csv'),
 WindowsPath('Input/VegasOdds/ncaabb13.csv'),
 WindowsPath('Input/VegasOdds/ncaabb14.csv'),
 WindowsPath('Input/VegasOdds/ncaabb15.csv'),
 WindowsPath('Input/VegasOdds/ncaabb16.csv'),
 WindowsPath('Input/Vegas

In [4]:
teams = pd.read_csv(data_dir+'Teams.csv')
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


In [5]:
team_spellings = pd.read_csv(data_dir+'TeamSpellings.csv')
team_spellings.sort_values('TeamID')
team_spelling_dict = dict(zip(team_spellings.TeamNameSpelling, team_spellings.TeamID))

In [6]:
def assign_team(team_string, team_spelling_dict=team_spelling_dict):
    '''
    Given a string of the team's name (team_string), return its TeamID from the dictionary of Team ID's and spellings
    '''
    team_string = team_string.lower()
    try:
        return team_spelling_dict[team_string]
    except KeyError:
        # Modify string if team_string initially isn't found
        try:
            team_string = team_string.replace(' ', '-')
            return team_spelling_dict[team_string]

        except KeyError:
            print("NOT FOUND:", team_string)
            return np.NaN
        
odds_2017 = pd.read_csv(vegas_dir+'ncaabb17.csv')

# Assign Team ID to Home/Road team
odds_2017['homeID'] = list(map(assign_team, odds_2017.home))
odds_2017['roadID'] = list(map(assign_team, odds_2017.road))

# Initial odds dataframe
print(odds_2017.shape)

# Drop teams that can't be found
odds_2017.dropna(subset=['homeID','roadID'], inplace=True)
odds_2017[['homeID', 'roadID']] = odds_2017[['homeID', 'roadID']].astype(int)
print(odds_2017.shape)
odds_2017.tail()

NOT FOUND: nj-tech
NOT FOUND: concordia-st.-paul
NOT FOUND: ut-rio-grande-valley
NOT FOUND: ut-rio-grande-valley
(4078, 33)
(4074, 33)


,date,home,hscore,road,rscore,line,lineavg,linesag,linesage,linesagp,...,linepib,line7ot,lineer,linedd,linemassey,linedunk,lineround,lineteamrnks,homeID,roadID
4073,3/30/2018,North Texas,88.0,San Francisco,77.0,4.5,2.63,1.49,10.89,1.53,...,NaN,-1.0,NaN,-6.68,5.27,-1.59,NaN,2.3,1317,1362
4074,3/30/2018,Northern Colorado,76.0,Illinois-Chicago,71.0,9.5,7.11,6.63,3.76,6.92,...,NaN,5.0,NaN,2.55,7.81,7.68,NaN,7.5,1294,1227
4075,3/31/2018,Michigan,69.0,Loyola-Chicago,57.0,5.5,6.59,7.59,0.78,7.65,...,NaN,2.0,NaN,2.79,6.05,8.99,NaN,7.1,1276,1260
4076,3/31/2018,Villanova,95.0,Kansas,79.0,5.0,4.60,3.72,3.99,3.67,...,NaN,2.0,NaN,1.11,6.03,6.06,NaN,4.6,1437,1242
4077,4/2/2018,Villanova,79.0,Michigan,62.0,6.5,4.79,6.19,-2.12,6.44,...,NaN,NaN,NaN,-2.34,5.35,2.97,NaN,7.0,1437,1276


In [7]:
odds_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4074 entries, 0 to 4077
Data columns (total 33 columns):
date            4074 non-null object
home            4074 non-null object
hscore          4066 non-null float64
road            4074 non-null object
rscore          4070 non-null float64
line            4055 non-null float64
lineavg         4074 non-null float64
linesag         4062 non-null float64
linesage        4062 non-null float64
linesagp        4062 non-null float64
linesaggm       4062 non-null float64
linemoore       4062 non-null float64
lineopen        4051 non-null float64
linedok         4067 non-null float64
linefox         4050 non-null float64
std             4074 non-null float64
linepugh        3991 non-null float64
linedonc        3958 non-null float64
neutral         4062 non-null float64
linetalis       3865 non-null float64
lineespn        3941 non-null float64
linepir         771 non-null float64
linepiw         0 non-null float64
linepib         0 non-null

In [8]:
odds_2017_basic = odds_2017[['date', 'home', 'homeID', 'hscore', 'road', 'roadID', 'rscore', 'lineavg']]
odds_2017_basic.to_csv('odds_2017_basic.csv')